In [ ]:
reviews = spark.sql("SELECT * FROM `default`.`reviews`")
restaurants = spark.sql("SELECT * FROM `default`.`us_restaurants`")

In [ ]:
us_restaurants = us_restaurants.withColumnRenamed("stars", "avg_star")
review = review.withColumnRenamed("stars", "review_star")

In [ ]:
# Merge the DataFrames on 'business_id'
restaurants_reviews = us_restaurants.join(review, on="business_id", how="inner")

In [ ]:
display(restaurants_reviews.limit(10))

In [ ]:
from pyspark.sql.functions import col, length, split, regexp_replace

# Remove '\n' characters and special characters from the 'text' column
restaurants_reviews = restaurants_reviews.withColumn("text_cleaned",
                                                     regexp_replace(col("text"), "[\\n!\"#$%&()*+,-./:;<=>?@[\\]^_`{|}~]", " "))


# Show the resulting DataFrame
display(restaurants_reviews.limit(10))

In [ ]:
restaurants_reviews = restaurants_reviews.withColumn("labels",
    when(restaurants_reviews["review_star"] >= 4, "positive")
    .when(restaurants_reviews["review_star"] == 3, "neutral")
    .when(restaurants_reviews["review_star"] < 3, "negative")
)

restaurants_reviews = restaurants_reviews.where(restaurants_reviews["labels"] != "neutral")

display(restaurants_reviews.limit(10))



In [ ]:
restaurants_reviews.write.option("database", "YelpDB").mode("overwrite").saveAsTable("restaurants_reviews")